In [1]:
!nvidia-smi

Thu Mar 11 01:24:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install flask-ngrok
!pip install flask-cors
!pip install transformers
!pip install -U sentence-transformers

     |████████████████████████████████| 1.9MB 18.6MB/s 
     |████████████████████████████████| 890kB 49.2MB/s 
     |████████████████████████████████| 3.2MB 46.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=e9898702a4b47e3147910ac9bd98879b419d4d850dde9ef2c1ecfe539aa8a1e5
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 71kB 8.7MB/s 
     |████████████████████████████████| 1.2MB 24.7MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.4.1.2-cp37-none-any.whl size=103068 sha256=2fe51302996e8742b222c29012066b5f3935a86b8c224fdd59edc000f378a981
  Stored in directory: /root/.cache/pip/wheels/3d/33/d1/5703dd56199c09d4a1b41e0c07fb4e7765a84d787cbdc48ac3
Successfully built sentence-transformers


In [3]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
def get_answer(questions, texts):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    answers = []
    scores = []
    for question in questions:
        for text in texts:

            inputs = tokenizer.encode_plus(question, text, add_special_tokens=True, padding=True, max_length=512, return_tensors="pt").to(device)
            offset_map = tokenizer.encode_plus(question, text, add_special_tokens=True, padding=True, max_length=512, return_tensors="pt", return_offsets_mapping=True)['offset_mapping'][0]
            input_ids = inputs["input_ids"].tolist()[0]
            
            text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
            model.to(device)
            res = model(**inputs)
            model.to('cpu')
            answer_start_scores = res['start_logits']
            answer_end_scores = res['end_logits']
            answer_start = torch.argmax(answer_start_scores)
            answer_end = torch.argmax(answer_end_scores) + 1
            score = torch.max(answer_start_scores) + torch.max(answer_end_scores)
            
            
            if answer_start >= answer_end - 1:
                continue

            if score < 0.75:
                continue
            

            start = offset_map[answer_start][0]
            end = offset_map[answer_end][1]
            # answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
            answer = text[start:end]
            if len(answer) == 0:
                continue
            answers.append(answer)
            if len(answers) > 10:
                return answers
    return answers

In [4]:
from flask import Flask 
from flask import request
from flask_ngrok import run_with_ngrok
from flask_cors import CORS, cross_origin
from flask import jsonify
from collections import defaultdict, Counter
from heapq import heappush, heappop


In [5]:
from collections import Counter, defaultdict
class URL:
    def __init__(self):
        # self.url = url
        self.asked_questions = defaultdict(set)
        self.counter = Counter()
        self.qa_sep = "#@"
        self.sep = "%$"

    def add_qa(self, question, answer):
        answer = answer.replace('\n', self.sep)
        self.asked_questions[question].add(answer)
        self.counter[question] += 1
        print("add_qa", answer)

    def get_top_qa(self, top=3):
        qas = sorted([(q, a) for q, a in self.asked_questions.items()], key=lambda x: -self.counter[x[0]])
        print(self.counter)
        return qas[:top]

    def get_answer(self, question):
        return self.asked_questions[question]

    def get_top_qa_string(self, top=3):
        print("top qa before string: ", self.get_top_qa(top))
        qa = self.qa_sep.join([q+self.sep+ self.sep.join(list(a)) for q, a in self.get_top_qa(top)])
        return qa


class Database:
    def __init__(self):
        self.urls = defaultdict(URL)

    def set_url_qa(self, url, question, answer):
        self.urls[url].add_qa(question, answer)

    def get_url(self, url):
        if url in self.urls:
            return self.urls[url]
        return None

    def get_url_top_qa(self, url, top=3):
        if url not in self.urls:
            return ""
        else:
            return self.urls[url].get_top_qa_string()

    def get_answer(self, url, question):
        url_obj = self.get_url(url)
        if url_obj is None:
            return ""


    def remove_url(self, url):
        if url not in self.urls:
            return
        del self.urls[url]

database = Database()

In [6]:
app = Flask(__name__) 
# CORS(app)
run_with_ngrok(app)    
app.config['CORS_HEADERS'] = 'Content-Type'
cors = CORS(app, resources={r"/api/*": {"origins": "*"}})
@app.route("/") 
def home(): 
    return "<h1>GFG is great platform to learn</h1>"

# @cross_origin(headers=['Content-Type']) # Send Access-Control-Allow-Headers
@app.route('/api/query', methods = ['POST', 'GET'])
@cross_origin(headers=['Content-Type'])
def get_query_from_react():
    sep = '%$'
    data = request.get_json()
    print(data.keys())
    print(data['data'])
    
    texts = data['data'].split(sep)
    print("texts", texts)
    question = texts[0]
    contents = [text for text in texts[1:] if len(text.split(' ')) > 20]
    print("contents", contents)
    is_answer = texts[-1] == "@answer"
    url = texts[-2]
    print("number of paragraphs: ", len(contents))
    print("Question is: ", question)
    print("Contents are: ", contents)
    print("URL is ", url)
    print("is answer", is_answer)
    answers = ""
    if not is_answer:
        old_answers = database.get_answer(url, question)
        if old_answers:
            answers = old_answers
        else:
            answers = sep.join(get_answer([question], contents))
    else: # get answer from front-end
        answers = sep.join(texts[1:-2])
    
    database.set_url_qa(url, question, answers)

    print(answers)
    print("Answer: ", answers)
    #r = "the"
    return jsonify(answers)

@app.route('/api/init', methods = ['POST', 'GET'])
@cross_origin(headers=['Content-Type'])
def get_top_question_answer():
    sep = '%$'
    data = request.get_json()
    print(data)
    url = data['data']
    print(url)
    qa = database.get_url_top_qa(url)
    print("top qa are: ", qa)
    print(jsonify(qa))
    return jsonify(qa)


In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://a798e872b508.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
